In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow_datasets as tfds

In [ ]:
# Set up GPU configuration for TensorFlow
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
else:
    print("No GPU found")

GPU found


In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess MNIST data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# Define CNN model for MNIST
def mnist_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
# Compile and train CNN model on MNIST dataset
model_mnist = mnist_cnn()
model_mnist.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_mnist.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Evaluate accuracy of CNN model on MNIST dataset
_, acc_mnist = model_mnist.evaluate(x_test, y_test)
print("Accuracy on MNIST dataset: {:.2f}%".format(acc_mnist * 100))

Epoch 1/10
469/469 [==============================] - 9s 9ms/step - loss: 0.2355 - accuracy: 0.9284 - val_loss: 0.0558 - val_accuracy: 0.9824
Epoch 2/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0820 - accuracy: 0.9760 - val_loss: 0.0387 - val_accuracy: 0.9858
Epoch 3/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0593 - accuracy: 0.9825 - val_loss: 0.0363 - val_accuracy: 0.9880
Epoch 4/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0485 - accuracy: 0.9851 - val_loss: 0.0310 - val_accuracy: 0.9896
Epoch 5/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0404 - accuracy: 0.9870 - val_loss: 0.0292 - val_accuracy: 0.9899
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0341 - accuracy: 0.9890 - val_loss: 0.0300 - val_accuracy: 0.9903
Epoch 7/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0287 - accuracy: 0.9904 - val_loss: 0.0295 - val_accuracy: 0.9908
Epoch 

In [ ]:
# Load SVHN dataset
svhn_train, svhn_test = tfds.load('svhn_cropped', split=['train', 'test'], shuffle_files=True, as_supervised=True)

In [ ]:
# Preprocess SVHN data
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0 
    return image, label


In [ ]:

# Use CNN model trained on MNIST as pre-trained model for SVHN
model_svhn  = Sequential()
model_svhn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model_svhn.add(Conv2D(64, (3, 3), activation='relu'))
model_svhn.add(MaxPooling2D(pool_size=(2, 2)))
model_svhn.add(Flatten())
model_svhn.add(Dense(128, activation='relu'))
model_svhn.add(Dropout(0.5))
model_svhn.add(Dense(10, activation='softmax'))

model_svhn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
svhn_train = svhn_train.map(preprocess_image).map(lambda x, y: (x, tf.one_hot(y, 10))).batch(128)
svhn_test = svhn_test.map(preprocess_image).map(lambda x, y: (x, tf.one_hot(y, 10))).batch(128)

# Copy weights from MNIST model to SVHN model for layers with the same name
for i in range(len(model_mnist.layers)):
    if model_mnist.layers[i].name == model_svhn.layers[i].name:
        model_svhn.layers[i].set_weights(model_mnist.layers[i].get_weights())
for images, label in svhn_train.take(1):
    print(images.shape)
    print(f'labrl {label.shape}')
for layer in model_svhn.layers[:-1]:
    layer.trainable = False
model_svhn.fit(svhn_train, epochs=10, validation_data=svhn_test)

# Evaluate accuracy of CNN model on SVHN dataset
_, acc_svhn = model_svhn.evaluate(svhn_test)
print("Accuracy on SVHN dataset: {:.2f}%".format(acc_svhn * 100))

(128, 32, 32, 3)
labrl (128, 10)
Epoch 1/10
573/573 [==============================] - 19s 29ms/step - loss: 1.4731 - accuracy: 0.4974 - val_loss: 0.7535 - val_accuracy: 0.7905
Epoch 2/10
573/573 [==============================] - 18s 32ms/step - loss: 0.8873 - accuracy: 0.7149 - val_loss: 0.6694 - val_accuracy: 0.8083
Epoch 3/10
573/573 [==============================] - 18s 31ms/step - loss: 0.8023 - accuracy: 0.7435 - val_loss: 0.6028 - val_accuracy: 0.8287
Epoch 4/10
573/573 [==============================] - 18s 32ms/step - loss: 0.7437 - accuracy: 0.7615 - val_loss: 0.5772 - val_accuracy: 0.8331
Epoch 5/10
573/573 [==============================] - 16s 29ms/step - loss: 0.7022 - accuracy: 0.7726 - val_loss: 0.5418 - val_accuracy: 0.8428
Epoch 6/10
573/573 [==============================] - 16s 28ms/step - loss: 0.6786 - accuracy: 0.7816 - val_loss: 0.5292 - val_accuracy: 0.8460
Epoch 7/10
573/573 [==============================] - 18s 32ms/step - loss: 0.6506 - accuracy: 0.7895 -